In [1]:

from moviepy.editor import VideoFileClip
from IPython.display import HTML

# output = "bullshit.mp4"
# clip1 = VideoFileClip("../original_clip.mp4")
# clip1.write_videofile(output, audio=False)


In [3]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon



In [7]:
ROOT_DIR = os.path.abspath("../Mask_RCNN/") 
sys.path.append(ROOT_DIR)
# sys.path.append(os.path.join(ROOT_DIR, "mrcnn/") )  # Add RCNN directory to path
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # Add coco to path


In [8]:
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize

%matplotlib inline 

In [9]:
# Import COCO config
import coco


class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

ModuleNotFoundError: No module named 'coco'

In [10]:
import cv2
